In [34]:
from pwn import *

ss = [set('a'), set('b'), set('c'), set('d')]

print(ss)

for i in range(32):
    # p = i % 4
    # p_2 = (i + 2) % 4
    ns = set()
    ns.add(i)
    # ss[p] = (ss[p].symmetric_difference(ss[p_2])).symmetric_difference(ns)
    ss[0], ss[1], ss[2], ss[3] = ss[1], ss[2], ss[3], ss[0].symmetric_difference(ss[2]).symmetric_difference(ns)

ss[2] = ss[2].symmetric_difference(ss[0])
ss[3] = ss[3].symmetric_difference(ss[1])
print(ss[0], ss[1], ss[2], ss[3])


[{'a'}, {'b'}, {'c'}, {'d'}]
{2, 4, 8, 10, 14, 16, 20, 'c', 22, 26, 28} {3, 5, 9, 11, 15, 17, 21, 'd', 23, 27, 29} {0, 2, 6, 8, 'a', 12, 14, 18, 20, 24, 26, 30} {1, 3, 7, 'b', 9, 13, 15, 19, 21, 25, 27, 31}


In [20]:
from typing import List
from Crypto.PublicKey import RSA
from Crypto.Util.number import bytes_to_long, long_to_bytes
from hashlib import sha256
import base64

key = RSA.generate(2048)
rsa_key = (key.n, key.e)
rsa_d = key.d


def packmess(mess):
    assert len(mess) <= 65535
    return mess+(len(mess).to_bytes(2, 'big'))


def unpackmess(mess):
    rlen = int.from_bytes(mess[-2:], 'big')
    if rlen > len(mess)-2:
        mess = b'\x00'*(rlen-len(mess)+2)+mess
    return mess[-(2+rlen):-2]


def getcert(name, key):
    rmess = packmess(name)+packmess(key)
    return enc(rmess)


def verify(cert):
    rmess = dec(cert)
    rkey = unpackmess(rmess)
    nmess = rmess[:-(len(rkey)+2)]
    rname = unpackmess(nmess)
    return (rname, rkey)


def enc(mess):
    gmess = int.from_bytes(mess, 'big')
    cip = pow(gmess, rsa_d, rsa_key[0])
    return cip


def dec(cip):
    mess = pow(cip, rsa_key[1], rsa_key[0])
    messm = mess.to_bytes((mess.bit_length()+7)//8, 'big')
    return messm


def pad(msg):
    n = 32 - len(msg) % 32
    return msg + bytes([n]) * n


def unpad(msg):
    assert len(msg) > 0 and len(msg) % 32 == 0
    n = msg[-1]
    assert 1 <= n <= 32
    assert msg[-n:] == bytes([n]) * n
    return msg[:-n]


def MESenc(mess: bytes, skey: bytes):
    assert len(skey) == 8*32
    keys = [bytes_to_long(skey[i*8:(i+1)*8]) for i in range(32)]
    assert len(mess) % 32 == 0
    cip = b""
    for it in range(0, len(mess), 32):
        pmess = mess[it:it+32]
        a = bytes_to_long(pmess[0:8])
        b = bytes_to_long(pmess[8:16])
        c = bytes_to_long(pmess[16:24])
        d = bytes_to_long(pmess[24:32])
        for key in keys:
            a, b, c, d = b, c, d, a ^ c ^ key
        a = long_to_bytes(a, 8)
        b = long_to_bytes(b, 8)
        c = long_to_bytes(c, 8)
        d = long_to_bytes(d, 8)
        cip += a+b+c+d
    return cip


P = 90217665064898209874945440415607645619739552209209829698859302616784804764535733761793578496057231124307702060703777047113648993244620436073319613409562635176198907683514151487667596066485528729421061091099275256268502121631878101098253283229895343146346757918637948217208979264467987857303672675468072062419
G = 13


In [47]:
sh = remote("prob08.geekgame.pku.edu.cn",10008)
r = sh.recv(numb = 2048, timeout = 2000)
# print(r)
token = "403:MEUCIQD3KO+wCR68OSEAc4JJYLR8Vtuwh//GTbDToFsQPITbuQIgXmMQ0znmP/DU2sKC0PX0ckbo779yoLYbKaO8pCbxgGo="
sh.sendline(token)
r = sh.recv(numb = 2048, timeout = 2000)
# print(r)
sh.sendline("0")
r = sh.recv(numb = 2048, timeout = 2000)
print(r)
sh.sendline("Alice".encode('utf-8').hex());
r = sh.recv(numb = 2048, timeout = 2000)
print(r)
sh.sendline("Key".encode('utf-8').hex());
r = sh.recv(numb = 2048, timeout = 2000)
print(r)

[x] Opening connection to prob08.geekgame.pku.edu.cn on port 10008
[x] Opening connection to prob08.geekgame.pku.edu.cn on port 10008: Trying 115.27.243.170
[+] Opening connection to prob08.geekgame.pku.edu.cn on port 10008: Done
b'My RSA N is: 23097941225098921027065992914841812285560391094743125078125375489283400049603070708843500375248080834959016833364944828325725303978369981509881550870065722163041736596999348682660751887448489878099228060526494814436021540974587483286458825581730327873105512224032648022148656115195432917541069297518976873089520491389431912769179263291658482366004166548189946459925744114395437151128415520669778987265646919440616572074720492212011604594102823169409116063017057531872839909040892696667849692932118552843976266357785696355382355039283667831119215759145487310184655662907346132986115865740337596886685605553106500319711\nMy RSA e is: 65537\nWhat is your name?\n'
b'What is your key?\n'
b'I can not issue multiple certificates to one person.\nWhat do you wan

<ipython-input-47-6ef009b89fed>:5: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  sh.sendline(token)
<ipython-input-47-6ef009b89fed>:8: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  sh.sendline("0")
<ipython-input-47-6ef009b89fed>:11: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  sh.sendline("Alice".encode('utf-8').hex());
<ipython-input-47-6ef009b89fed>:14: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  sh.sendline("Key".encode('utf-8').hex());


In [39]:
import binascii 

def decode(mess: bytes, cip: bytes, flag: bytes):
    skey = [0] * 32
    assert len(mess) % 32 == 0
    ans = b''
    for it in range(0, len(mess), 32):
        pmess = mess[it:it+32]
        pcip = cip[it:it+32]
        pflag = flag[it:it+32]
        a0 = bytes_to_long(pmess[0:8])
        b0 = bytes_to_long(pmess[8:16])
        c0 = bytes_to_long(pmess[16:24])
        d0 = bytes_to_long(pmess[24:32])
        a1 = bytes_to_long(pcip[0:8])
        b1 = bytes_to_long(pcip[8:16])
        c1 = bytes_to_long(pcip[16:24])
        d1 = bytes_to_long(pcip[24:32])
        c1 ^= a1
        d1 ^= b1
        k1 = c0 ^ a1
        k2 = d0 ^ b1
        k3 = a0 ^ c1
        k4 = b0 ^ d1
        a1 ^= k1
        b1 ^= k2
        c1 ^= k3
        d1 ^= k4
        print(k1, k2, k3, k4)
        
        a = long_to_bytes(c1, 8)
        b = long_to_bytes(d1, 8)
        c = long_to_bytes(a1, 8)
        d = long_to_bytes(b1, 8)
        ans += a+b+c+d
    print(ans)
    ans = b''
    for it in range(0, len(flag), 32):
        pflag = flag[it:it+32]
        a2 = bytes_to_long(pflag[0:8])
        b2 = bytes_to_long(pflag[8:16])
        c2 = bytes_to_long(pflag[16:24])
        d2 = bytes_to_long(pflag[24:32])
        c2 ^= a2
        d2 ^= b2
        a2 ^= k1
        b2 ^= k2
        c2 ^= k3
        d2 ^= k4
        a = long_to_bytes(c2, 8)
        b = long_to_bytes(d2, 8)
        c = long_to_bytes(a2, 8)
        d = long_to_bytes(b2, 8)
        ans += a+b+c+d
    return ans

tmp = r.decode().split('\n')
tmp[0] = binascii.a2b_hex(tmp[0])
tmp[1] = binascii.a2b_hex(tmp[1])

# Public_key=os.urandom(256)
# MESenc(pad(("Hello, Alice! I will give you two flags. The first is: "+flag1(token)).encode("utf-8")),Public_key).hex()
padded = pad(("Sorry, I forget to verify your identity. Please give me your certificate.").encode("utf-8"))
decode(padded, tmp[1], tmp[0])

17772651477269818341 12667178601735605006 16254714827484711362 10003855788647640641
17772651477269818341 12667178601735605006 16254714827484711362 10003855788647640641
17772651477269818341 12667178601735605006 16254714827484711362 10003855788647640641
b'Sorry, I forget to verify your identity. Please give me your certificate.\x17\x17\x17\x17\x17\x17\x17\x17\x17\x17\x17\x17\x17\x17\x17\x17\x17\x17\x17\x17\x17\x17\x17'


b'Hello, Alice! I will give you two flags. The first is: flag{Fe1STeL_neTw0rk_ne3d_an_OWf}\x08\x08\x08\x08\x08\x08\x08\x08'

In [50]:
b'\x00'*-5 + b'asdf'

b'asdf'

In [54]:
ttt = "`\"\b"
print(ttt)

`
